# Loading the data

In [3]:
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
#%matplotlib notebook
%matplotlib widget
#This make the plot interactive

a = pd.read_csv('datos.txt',header=None)
#a = pd.read_csv('datoscompletos.txt',header=None)
data = a.values #numpy array
trainingexamples, numberfeaturesy = data.shape #It can be a tuple or not
X = data[:, 0:(numberfeaturesy-1)] #The interval are always opened interval
y = data[:, (numberfeaturesy-1)]
(m,) = y.shape

#Showing the data
fig = plt.figure(1)
#ax = fig.add_subplot(111, projection='3d')
ax = fig.gca(projection='3d')
ax.scatter(X[:,0], X[:,1], y, c = 'r', marker = 'o')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Scale features and set the to zero mean

In [4]:
def FeatureNormalize(X):
    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0, ddof=1)
    X_norm = (X-mu)/sigma
    return X_norm, mu, sigma

In [5]:
X, mu, sigma = FeatureNormalize(X)
X = np.hstack((np.ones((m,1)), X))

#Showing the data
fig = plt.figure(2)
#ax = fig.add_subplot(111, projection='3d')
ax = fig.gca(projection='3d')
ax.scatter(X[:,0], X[:,1], y, c = 'r', marker = 'o')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Gradiente Descent

In [6]:
#Choose some alpha value
alpha = 0.1
num_iters = 400

In [7]:
def GradientDescentMulti(X, y, theta, alpha, num_iters):
    (m,) = y.shape
    J_history = np.zeros((num_iters))
    for iter in range(num_iters):
        h = np.dot(X, theta)
        theta = theta - alpha*(1/m)*np.dot(X.T, h-y)
        J_history[iter] = ComputeCostMulti(X, y, theta)
        
    return theta, J_history

In [8]:
def ComputeCostMulti(X, y, theta):
    (m,) = y.shape
    h = np.dot(X, theta)
    DiferenciaCuadrados = (h-y)**2
    J = (1/(2*m))*np.sum(DiferenciaCuadrados)
    
    return J

In [9]:
#Init theta and Run Gradient Descent
theta = np.zeros((numberfeaturesy))
theta, J_history = GradientDescentMulti(X, y, theta, alpha, num_iters)
print(theta)

[110.857      -40.40139839 -32.24490408]


# Plotting the results

In [10]:
tx = np.linspace(-3, 3, 100)
ty = np.linspace(-3, 3, 100)
xx, yy = np.meshgrid(tx, ty)
tz = theta[0]+theta[1]*xx+theta[2]*yy

#Plot of the surface
fig = plt.figure(3)
#ax = fig.add_subplot(111, projection='3d')
ax = fig.gca(projection='3d')
ax.plot_surface(tx, ty, tz)
#ax.plot_wireframe(tx, ty, tz)
ax.set_xlabel('tx')
ax.set_ylabel('ty')
plt.show()

#Plot cost function
fig = plt.figure(4)
plt.plot(np.arange(1, num_iters+1, 1), J_history)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
X_example = np.array([0.82, 315]) #This is a prediction
X_example = (X_example-mu)/sigma
o = np.array([1])
X_example = np.hstack((o, X_example))
#X_example = np.insert(X, 0, 1)
price = np.dot(X_example, theta)
print(price)
X = np.hstack((np.ones((m,1)), X))

123.04286652664086


# Importing all the excel files and exporting to csv files

### Precio de Oferta

In [12]:
#Para introducir esta data debo ante incluir la columna Capacidad Efectiva Neta (MW)
l = pd.read_excel('Oferta2017.xlsx') #read the excel file
l = l.iloc[2:,:] #reshape the table for the position [2:,:,-2]
colum = l.iloc[0].values #This are the columns headers
l.columns = colum #Making the columns of the dataframe to be the columns we obtain in the previous step
l.drop([2], axis=0, inplace=True) #Eliminate fron the dataframe the 2 rown which is the columns header
l.dropna(inplace=True)
l.reset_index(drop=True, inplace=True) #I make a reset to all the row indexis
l.drop(['Código Agente', 'Precio de Oferta Ideal $/kWh', 'Precio de Oferta de Despacho $/kWh'], axis=1, inplace=True)
l.to_csv('Oferta2017.csv', index=False)
l

Fecha     Recurso Precio de Oferta Declarado $/kWh  \
0     2017-01-01       ALBAN                              290   
1     2017-01-01     BETANIA                          57.0771   
2     2017-01-01      CALIMA                            294.5   
3     2017-01-01      CHIVOR                           88.999   
4     2017-01-01   EL QUIMBO                              300   
...          ...         ...                              ...   
7295  2017-12-31       PRADO                              110   
7296  2017-12-31   SALVAJINA                              150   
7297  2017-12-31  SAN CARLOS                              145   
7298  2017-12-31    SOGAMOSO                              169   
7299  2017-12-31        URRA                           58.926   

     Capacidad Efectiva Neta (MW)  
0                             429  
1                             540  
2                             132  
3                            1000  
4                             400  
...                           ...  
7295                           51  
7296                          315  
7297                         1240  
7298                          819  
7299                          338  

[7300 rows x 4 columns]

### Relacion entre Generador y Embalse

In [13]:
l = pd.read_excel('Generador_Embalse.xlsx') #read the excel file
index = l[l['Embalse']=='NO APLICA'].index #iloc for numeric indexes matrix and loc as header tables and without loc or iloc always enter to columns
l.drop(index, axis=0, inplace=True)
l.reset_index(drop=True, inplace=True)
#index = l[l['Embalse']=='AGREGADO BOGOTA (SISGA-NEUSA-TOMINE)'].index
#idx = l[l['Embalse'].str.contains('AGREGADO BOGOTA \(SISGA-NEUSA-TOMINE\)', regex=True)].index
#l.loc[idx, 'Embalse'] = 'AGREGADO BOGOTA'
l.loc[[0, 5, 12], 'Embalse'] = 'ALTOANCHICAYA', 'PENOL', 'AGREGADO BOGOTA'
l.to_csv('Generador_Embalse.csv', index=False)
l

Central (planta) Hidráulica          Embalse
0                        ALBAN    ALTOANCHICAYA
1                      BETANIA          BETANIA
2                       CALIMA          CALIMA1
3                       CHIVOR        ESMERALDA
4                    EL QUIMBO        EL QUIMBO
5                      GUATAPE            PENOL
6                      GUATRON         TRONERAS
7                      GUATRON       MIRAFLORES
8                       GUAVIO           GUAVIO
9                       JAGUAS      SAN LORENZO
10                 LA TASAJERA       RIOGRANDE2
11                      MIEL I            AMANI
12                       PAGUA  AGREGADO BOGOTA
13                       PAGUA             MUNA
14                       PAGUA            CHUZA
15                      PLAYAS           PLAYAS
16                    PORCE II         PORCE II
17                   PORCE III        PORCE III
18                       PRADO            PRADO
19                   SALVAJINA        SALVAJINA
20                  SAN CARLOS         PUNCHINA
21                    SOGAMOSO         TOPOCORO
22                        URRA            URRA1

### Reserva

In [14]:
#Para introducir esta data debo ante incluir la columna Volumen Maximo Util Gwh
l = pd.read_excel('Reserva2017.xlsx') #read the excel file
l = l.iloc[2:,:]
colums = l.iloc[0].values
l.columns = colums
l.drop([2], axis=0, inplace=True)
l.reset_index(drop=True, inplace=True)
l.drop(['Region Hidrologica', 'Volumen Útil Diario Mm3', 'Volumen Útil Díario %', 'Volumen Mm3', 'Volumen Energía kWh', 'Volumen %'], axis=1, inplace=True)
l.to_csv('Reserva2017.csv', index=False)
l

Fecha Nombre Embalse Volumen Útil Diario Energía kWh  \
0     2017-01-01          AMANI                       239250900   
1     2017-01-01     MIRAFLORES                       262356900   
2     2017-01-01          PENOL                      3660319800   
3     2017-01-01         PLAYAS                       104566800   
4     2017-01-01       PORCE II                        94036400   
...          ...            ...                             ...   
8390  2017-12-31      ESMERALDA                       894784700   
8391  2017-12-31         GUAVIO                      1448190900   
8392  2017-12-31  ALTOANCHICAYA                          542600   
8393  2017-12-31        CALIMA1                       157927800   
8394  2017-12-31      SALVAJINA                       102868800   

     Volumen Máximo Util Gwh  
0                     246.13  
1                      313.1  
2                    4078.46  
3                      96.35  
4                     130.14  
...                      ...  
8390                 1112.97  
8391                 2091.46  
8392                   36.88  
8393                  217.04  
8394                  171.82  

[8395 rows x 4 columns]

# Trabajar con los .csv creados

In [17]:
import datetime

In [18]:
reserva = pd.read_csv('Reserva2017.csv')
generador_embalse = pd.read_csv('Generador_Embalse.csv')
oferta = pd.read_csv('Oferta2017.csv')

In [19]:
generador_embalse

Central (planta) Hidráulica          Embalse
0                        ALBAN    ALTOANCHICAYA
1                      BETANIA          BETANIA
2                       CALIMA          CALIMA1
3                       CHIVOR        ESMERALDA
4                    EL QUIMBO        EL QUIMBO
5                      GUATAPE            PENOL
6                      GUATRON         TRONERAS
7                      GUATRON       MIRAFLORES
8                       GUAVIO           GUAVIO
9                       JAGUAS      SAN LORENZO
10                 LA TASAJERA       RIOGRANDE2
11                      MIEL I            AMANI
12                       PAGUA  AGREGADO BOGOTA
13                       PAGUA             MUNA
14                       PAGUA            CHUZA
15                      PLAYAS           PLAYAS
16                    PORCE II         PORCE II
17                   PORCE III        PORCE III
18                       PRADO            PRADO
19                   SALVAJINA        SALVAJINA
20                  SAN CARLOS         PUNCHINA
21                    SOGAMOSO         TOPOCORO
22                        URRA            URRA1

In [20]:
#We make the 'Fecha' column a datime series
reserva['Fecha'] = pd.to_datetime(reserva['Fecha'])
oferta['Fecha'] = pd.to_datetime(oferta['Fecha'])

In [21]:
oferta['Volumen Util'] = ''
oferta

Fecha     Recurso  Precio de Oferta Declarado $/kWh  \
0    2017-01-01       ALBAN                         290.00000   
1    2017-01-01     BETANIA                          57.07706   
2    2017-01-01      CALIMA                         294.50000   
3    2017-01-01      CHIVOR                          88.99900   
4    2017-01-01   EL QUIMBO                         300.00000   
...         ...         ...                               ...   
7295 2017-12-31       PRADO                         110.00000   
7296 2017-12-31   SALVAJINA                         150.00000   
7297 2017-12-31  SAN CARLOS                         145.00000   
7298 2017-12-31    SOGAMOSO                         169.00000   
7299 2017-12-31        URRA                          58.92603   

      Capacidad Efectiva Neta (MW) Volumen Util  
0                              429               
1                              540               
2                              132               
3                             1000               
4                              400               
...                            ...          ...  
7295                            51               
7296                           315               
7297                          1240               
7298                           819               
7299                           338               

[7300 rows x 5 columns]

In [22]:
reserva['Generador'] = ''
reserva

Fecha Nombre Embalse  Volumen Útil Diario Energía kWh  \
0    2017-01-01          AMANI                     2.392509e+08   
1    2017-01-01     MIRAFLORES                     2.623569e+08   
2    2017-01-01          PENOL                     3.660320e+09   
3    2017-01-01         PLAYAS                     1.045668e+08   
4    2017-01-01       PORCE II                     9.403640e+07   
...         ...            ...                              ...   
8390 2017-12-31      ESMERALDA                     8.947847e+08   
8391 2017-12-31         GUAVIO                     1.448191e+09   
8392 2017-12-31  ALTOANCHICAYA                     5.426000e+05   
8393 2017-12-31        CALIMA1                     1.579278e+08   
8394 2017-12-31      SALVAJINA                     1.028688e+08   

      Volumen Máximo Util Gwh Generador  
0                      246.13            
1                      313.10            
2                     4078.46            
3                       96.35            
4                      130.14            
...                       ...       ...  
8390                  1112.97            
8391                  2091.46            
8392                    36.88            
8393                   217.04            
8394                   171.82            

[8395 rows x 5 columns]

In [23]:
for x in tqdm(generador_embalse['Embalse'].values):
    idx1 = generador_embalse[generador_embalse['Embalse']==x].index
    idx2 = reserva[reserva['Nombre Embalse']==x].index
    reserva.loc[idx2, 'Generador'] = generador_embalse.loc[idx1, 'Central (planta) Hidráulica'].values
    
reserva

Fecha Nombre Embalse  Volumen Útil Diario Energía kWh  \
0    2017-01-01          AMANI                     2.392509e+08   
1    2017-01-01     MIRAFLORES                     2.623569e+08   
2    2017-01-01          PENOL                     3.660320e+09   
3    2017-01-01         PLAYAS                     1.045668e+08   
4    2017-01-01       PORCE II                     9.403640e+07   
...         ...            ...                              ...   
8390 2017-12-31      ESMERALDA                     8.947847e+08   
8391 2017-12-31         GUAVIO                     1.448191e+09   
8392 2017-12-31  ALTOANCHICAYA                     5.426000e+05   
8393 2017-12-31        CALIMA1                     1.579278e+08   
8394 2017-12-31      SALVAJINA                     1.028688e+08   

      Volumen Máximo Util Gwh  Generador  
0                      246.13     MIEL I  
1                      313.10    GUATRON  
2                     4078.46    GUATAPE  
3                       96.35     PLAYAS  
4                      130.14   PORCE II  
...                       ...        ...  
8390                  1112.97     CHIVOR  
8391                  2091.46     GUAVIO  
8392                    36.88      ALBAN  
8393                   217.04     CALIMA  
8394                   171.82  SALVAJINA  

[8395 rows x 5 columns]

In [24]:
tiempo = pd.unique(reserva['Fecha'])
generadores = generador_embalse['Central (planta) Hidráulica'].values
for x in tqdm(tiempo):
    index1 = reserva[reserva['Fecha']==x]#indexis for reserva
    index2 = oferta[oferta['Fecha']==x]#indexis for oferta
    for y in generadores:
        
        ind1 = index1[index1['Generador']==y].index
        volumenutil = (reserva.loc[ind1, 'Volumen Útil Diario Energía kWh'].sum()/1000000)/reserva.loc[ind1, 'Volumen Máximo Util Gwh'].sum()

        ind2 = index2[index2['Recurso']==y].index
        oferta.loc[ind2, 'Volumen Util'] = volumenutil

    
oferta
    

Fecha     Recurso  Precio de Oferta Declarado $/kWh  \
0    2017-01-01       ALBAN                         290.00000   
1    2017-01-01     BETANIA                          57.07706   
2    2017-01-01      CALIMA                         294.50000   
3    2017-01-01      CHIVOR                          88.99900   
4    2017-01-01   EL QUIMBO                         300.00000   
...         ...         ...                               ...   
7295 2017-12-31       PRADO                         110.00000   
7296 2017-12-31   SALVAJINA                         150.00000   
7297 2017-12-31  SAN CARLOS                         145.00000   
7298 2017-12-31    SOGAMOSO                         169.00000   
7299 2017-12-31        URRA                          58.92603   

      Capacidad Efectiva Neta (MW) Volumen Util  
0                              429     0.353078  
1                              540     0.728371  
2                              132     0.605423  
3                             1000     0.843577  
4                              400     0.618243  
...                            ...          ...  
7295                            51     0.774983  
7296                           315     0.598701  
7297                          1240     0.578973  
7298                           819     0.789005  
7299                           338      0.67847  

[7300 rows x 5 columns]

In [386]:
dpi = pd.to_datetime('2017-01-02')
index = reserva[reserva['Fecha']==dpi]
ind = index[index['Generador']=='GUATRON'].index
suma1= reserva.loc[ind, 'Volumen Máximo Util Gwh'].sum()
suma2 = reserva.loc[ind, 'Volumen Útil Diario Energía kWh'].sum()
diff = (suma2/1000000)/suma1
oferta[oferta['Fecha']==dpi]

Fecha      Recurso  Precio de Oferta Declarado $/kWh  \
20 2017-01-02        ALBAN                         100.00000   
21 2017-01-02      BETANIA                          90.00000   
22 2017-01-02       CALIMA                         294.50000   
23 2017-01-02       CHIVOR                          57.07703   
24 2017-01-02    EL QUIMBO                         200.00000   
25 2017-01-02      GUATAPE                          57.07701   
26 2017-01-02      GUATRON                         160.00000   
27 2017-01-02       GUAVIO                          57.07705   
28 2017-01-02       JAGUAS                          57.07704   
29 2017-01-02  LA TASAJERA                          57.07706   
30 2017-01-02       MIEL I                          57.07702   
31 2017-01-02        PAGUA                         200.00001   
32 2017-01-02       PLAYAS                          57.07700   
33 2017-01-02     PORCE II                          74.00000   
34 2017-01-02    PORCE III                          78.00000   
35 2017-01-02        PRADO                         120.00000   
36 2017-01-02    SALVAJINA                         294.00000   
37 2017-01-02   SAN CARLOS                          85.00000   
38 2017-01-02     SOGAMOSO                          65.00000   
39 2017-01-02         URRA                          57.07800   

    Capacidad Efectiva Neta (MW) Volumen Util  
20                           429      0.46308  
21                           540     0.756518  
22                           132     0.608311  
23                          1000     0.836117  
24                           400     0.623865  
25                           560     0.897196  
26                           512     0.806329  
27                          1250     0.760073  
28                           170     0.932267  
29                           306     0.789632  
30                           396     0.963691  
31                           600     0.513675  
32                           207      1.08716  
33                           405     0.666443  
34                           700     0.706849  
35                            51     0.990788  
36                           315       0.8166  
37                          1240     0.698538  
38                           819     0.923786  
39                           338     0.959957

# Organize data.txt

In [29]:
datos = Informacion_datos[['Volumen Util', 'Capacidad Efectiva Neta (MW)', 'Precio de Oferta Declarado $/kWh']].copy()
datos

Volumen Util  Capacidad Efectiva Neta (MW)  \
0         0.353078                           429   
1         0.728371                           540   
2         0.605423                           132   
3         0.843577                          1000   
4         0.618243                           400   
...            ...                           ...   
7295      0.774983                            51   
7296      0.598701                           315   
7297      0.578973                          1240   
7298      0.789005                           819   
7299      0.678470                           338   

      Precio de Oferta Declarado $/kWh  
0                            290.00000  
1                             57.07706  
2                            294.50000  
3                             88.99900  
4                            300.00000  
...                                ...  
7295                         110.00000  
7296                         150.00000  
7297                         145.00000  
7298                         169.00000  
7299                          58.92603  

[7300 rows x 3 columns]

In [30]:
#datos.to_csv('datoscompletos.txt', index=False, header=False)
datos.to_csv('datoscompletos.txt', index=False, header=True)